In [1]:
import os
import sys
import joblib
import torch
import trimesh
import pickle
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
from PIL import Image, ImageOps
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display

sys.path.append('/home/sid/Projects/reprojection')
sys.path.append('/home/sid/Projects/humor/humor')

from body_model.body_model import BodyModel
from humor_inference.reproject_humor_sequence import get_camera_params, make_44
from reproject.reproject_mesh.run_reprojection import (
    render_mesh, 
    get_meshes,
    get_synced_meshes,
    render_mesh,
    transform_meshes 
)
from fitting.fitting_utils import run_smpl
from humor_inference.reproject_wham_sequence import sanitize_wham_preds, get_wham_mesh_sequence, get_wham_parameters, transform_mesh_sequence
from reproject.reproject_mesh.run_reprojection_wham import MeshSynchronizer, MeshProcessor, Renderer
from reproject.reproject_mesh.reprojection_utils import get_calib_paths

In [2]:
def visualize_coordinate_system(mesh, transform_option='identity', save_path=None):
    """
    Visualize mesh with different coordinate transformations.
    
    Args:
        mesh (trimesh.Trimesh): Input mesh to visualize
        transform_option (str): Transformation to apply:
            - 'identity': No transformation
            - 'invert_yz': Invert Y and Z axes
            - 'invert_xz': Invert X and Z axes
            - 'rotate_y_180': Rotate 180° around Y
            - 'swap_yz': Swap Y and Z axes
            - 'invert_z': Invert Z-axis
        save_path (str, optional): Path to save visualization
    """
    # Create transformation matrix
    if transform_option == 'identity':
        alignment_matrix = np.eye(4)
    elif transform_option == 'invert_yz':
        alignment_matrix = np.eye(4)
        alignment_matrix[1, 1] = -1
        alignment_matrix[2, 2] = -1
    elif transform_option == 'invert_xz':
        alignment_matrix = np.eye(4)
        alignment_matrix[0, 0] = -1
        alignment_matrix[2, 2] = -1
    elif transform_option == 'rotate_y_180':
        alignment_matrix = trimesh.transformations.rotation_matrix(np.pi, [0, 1, 0])
    elif transform_option == 'swap_yz':
        alignment_matrix = np.array([
            [1, 0, 0, 0], 
            [0, 0, 1, 0],
            [0, 1, 0, 0],
            [0, 0, 0, 1],
        ])
    elif transform_option == 'invert_z':
        alignment_matrix = np.eye(4)
        alignment_matrix[2, 2] = -1
    else:
        raise ValueError(f"Unknown option '{transform_option}'")

    # Apply transformation
    mesh = mesh.copy()
    mesh.apply_transform(alignment_matrix)

    # Visualization
    fig = plt.figure(figsize=(15, 5))
    views = [(0, 0), (90, 0), (0, 90)]
    titles = ['Front (X-Y)', 'Side (Y-Z)', 'Top (X-Z)']
    
    for i, (elev, azim) in enumerate(views, 1):
        ax = fig.add_subplot(1, 3, i, projection='3d')
        
        # Plot vertices
        vertices = mesh.vertices
        ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], 
                  c='b', marker='.', s=1, alpha=0.1)
        
        # Plot axes
        origin = mesh.centroid
        axis_length = np.max(np.ptp(vertices, axis=0)) * 0.2
        
        ax.quiver(origin[0], origin[1], origin[2], 
                 axis_length, 0, 0, color='r', label='X')
        ax.quiver(origin[0], origin[1], origin[2], 
                 0, axis_length, 0, color='g', label='Y')
        ax.quiver(origin[0], origin[1], origin[2], 
                 0, 0, axis_length, color='b', label='Z')
        
        ax.set_title(f"{titles[i-1]}\n{transform_option}")
        ax.view_init(elev=elev, azim=azim)
        ax.legend()
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()
    
    # Print mesh statistics
    print(f"\nMesh Statistics ({transform_option}):")
    print(f"Centroid: {mesh.centroid}")
    print(f"Bounds: {mesh.bounds}")
    print(f"Up vector (Z) mean: {mesh.vertices[:, 2].mean():.3f}")
    return mesh


In [ ]:
HUMOR_OUT_FILE = "/home/NAS-mountpoint/kinect-omni-ego/2023-02-09/at-unis/lab/a04-2/capture0/out_capture0/results_out/final_results/stage3_results.npz"
WHAM_OUT_FILE = '/home/sid/Projects/WHAM/output/demo/2023-02-09_at-unis_lab_a01_capture0/wham_output.pkl'
BM_PATH = "/home/sid/Projects/humor/body_models/smplh/male/model.npz"

humor_output = np.load(HUMOR_OUT_FILE)
wham_output = joblib.load(WHAM_OUT_FILE)

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize BodyModel with 22 betas
bm = BodyModel(
    bm_path=BM_PATH,
    num_betas=22,  # Changed to match SMPLH shape space
    batch_size=1,
    model_type='smplh'
).to(device)

# Load WHAM data and pad betas to 22
wham_data = wham_output['results']
wham_betas = torch.tensor(wham_data[0]["betas"][0], device=device).float()  # [10]
wham_betas = torch.cat([
    wham_betas,
    torch.zeros(12, device=device)  # Pad to 22 dims
])

# Rest of WHAM processing
wham_pose = torch.tensor(wham_data[0]["pose_world"][0], device=device).float()
wham_trans = torch.tensor(wham_data[0]["trans_world"][0], device=device).float()

# Split pose correctly for SMPLH
root_orient = wham_pose[:3].reshape(1, -1)     # [1, 3]
pose_body = wham_pose[3:66].reshape(1, -1)     # [1, 63]
hand_pose = torch.zeros(1, 90, device=device)  # [1, 90]

# Pass to BodyModel with padded betas
try:
    wham_body = bm(
        pose_body=pose_body,        
        root_orient=root_orient,    
        pose_hand=hand_pose,        
        trans=trans,                
        betas=wham_betas.view(1, -1)  # Now [1, 16]
    )
except RuntimeError as e:
    print("Error during BodyModel forward pass:", e)
    raise

# Create Trimesh object for WHAM mesh
wham_mesh = trimesh.Trimesh(
    vertices=wham_body.v.detach().cpu().numpy()[0],
    faces=bm.bm.faces_tensor.cpu().numpy()  # Changed from bm.f to bm.bm.faces_tensor
)

# Load and process HuMoR mesh
# Load HuMoR results
humor_res = np.load(HUMOR_OUT_FILE)

# Convert HuMoR parameters to tensors and pad betas to 16 dimensions
humor_betas = torch.tensor(humor_res["betas"][0], device=device).float()      # [10]
humor_betas = torch.cat([                                                     # Pad to [16]
    humor_betas,
    torch.zeros(6, device=device)
])
humor_trans = torch.tensor(humor_res["trans"][0], device=device).float()      # [3]
humor_pose = torch.tensor(humor_res["pose_body"][0], device=device).float()   # [63]
humor_root = torch.tensor(humor_res["root_orient"][0], device=device).float() # [3]

# Initialize zero hand pose
humor_hand_pose = torch.zeros(1, 90, device=device)  # [1, 90] for SMPLH hands

# Get SMPL output for HuMoR parameters with padded betas
humor_body = bm(
    pose_body=humor_pose.view(1, -1),     # [1, 63]
    root_orient=humor_root.view(1, -1),    # [1, 3] 
    pose_hand=humor_hand_pose,            # [1, 90]
    trans=humor_trans.view(1, -1),        # [1, 3]
    betas=humor_betas.view(1, -1)         # [1, 16] - now padded
)
# Create trimesh from HuMoR SMPL output
humor_mesh = trimesh.Trimesh(
    vertices=humor_body.v.detach().cpu().numpy()[0],
    faces=bm.bm.faces_tensor.cpu().numpy()
)

# Compare coordinate systems
print("WHAM Original:")
wham_orig = visualize_coordinate_system(wham_mesh, 'identity')

print("\nHuMoR Original:")
humor_orig = visualize_coordinate_system(humor_mesh, 'identity')

# Apply transformations
transforms = ['invert_z', 'invert_yz', 'swap_yz']
for transform in transforms:
    print(f"\nTrying {transform}:")
    visualize_coordinate_system(wham_mesh, transform)

In [4]:
def process_mesh_data(
    mesh_type: str,  # Just "wham" or "humor"
    input_file: str,
    bm: 'BodyModel', 
    device: torch.device
) -> trimesh.Trimesh:
    """Process mesh data for either WHAM or HuMoR outputs."""
    if mesh_type.lower() not in ["wham", "humor"]:
        raise ValueError("mesh_type must be either 'wham' or 'humor'")
        
    # Rest of function remains same but with string comparison
    if mesh_type.lower() == "wham":
        data = joblib.load(input_file)
        betas = torch.tensor(data[0]["betas"][0], device=device).float()
        pose = torch.tensor(data[0]["pose_world"][0], device=device).float()
        trans = torch.tensor(data[0]["trans_world"][0], device=device).float()
        beta_padding = 12  # Pad from 10 to 22
    elif mesh_type.lower() == "humor":
        data = np.load(input_file)
        betas = torch.tensor(data["betas"][0], device=device).float()
        pose = torch.tensor(data["pose_body"][0], device=device).float()
        trans = torch.tensor(data["trans"][0], device=device).float()
        root_orient = torch.tensor(data["root_orient"][0], device=device).float()
        beta_padding = 6  # Pad from 16 to 22
    else:
        raise ValueError("Unknown mesh type")

    # Common processing
    betas = torch.cat([betas, torch.zeros(beta_padding, device=device)])
    hand_pose = torch.zeros(1, 90, device=device)
    
    # Handle poses differently based on type
    if mesh_type == "wham":
        root_orient = pose[:3].reshape(1, -1)
        pose_body = pose[3:66].reshape(1, -1)
    else:
        root_orient = root_orient.view(1, -1)
        pose_body = pose.view(1, -1)
    
    # Get body output
    try:
        body = bm(
            pose_body=pose_body,
            root_orient=root_orient,
            pose_hand=hand_pose,
            trans=trans.view(1, -1),
            betas=betas.view(1, -1)
        )
    except RuntimeError as e:
        print(f"Error processing {mesh_type} mesh:", e)
        raise
        
    # Create and return mesh
    return trimesh.Trimesh(
        vertices=body.v.detach().cpu().numpy()[0],
        faces=bm.bm.faces_tensor.cpu().numpy()
    )

# Example usage
# FOLDER_CHOSEN = "/home/NAS-mountpoint/kinect-omni-ego/2023-02-09/at-unis/lab/a04-2/capture0/"
# HUMOR_MESH = os.path.join(FOLDER_CHOSEN, "out_capture0/results_out/final_results/stage3_results.npz")
# BM_PATH = "/home/sid/Projects/humor/body_models/smplh/male/model.npz"
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Initialize BodyModel with 22 betas
# bm = BodyModel(
#     bm_path=BM_PATH,
#     num_betas=22,  # Changed to match SMPLH shape space
#     batch_size=1,
#     model_type='smplh'
# ).to(device)
# humor_mesh = process_mesh_data("humor", HUMOR_MESH, bm, device)

In [ ]:
# Script that uses the saved reprojected HuMoR meshes directly to render them on the omni images. 

ROOT_DIR = "/home/NAS-mountpoint/kinect-omni-ego/2024-01-12/at-unis/lab/sid/capture0"

LATERAL_VIEW_IMAGES = os.path.join(ROOT_DIR, "rgb")
OMNI_IMAGES = os.path.join(os.path.dirname(ROOT_DIR), "omni")
print(f"Root directory: {ROOT_DIR}")
print(f"Lateral view images: {LATERAL_VIEW_IMAGES}")
print(f"Omni images: {OMNI_IMAGES}")

MESH_DIR = os.path.join(ROOT_DIR, "out_capture0", "results_out", "final_results")
HUMOR_MESH_FILE = os.path.join(MESH_DIR, "stage3_results.npz")
REPROJECTED_MESHES_FOLDER = os.path.join(ROOT_DIR, "out_capture0", "reprojected", "meshes")

# Render the reprojected meshes onto omni images using intrinsics.
OMNI_CALIB_FILE = "/home/sid/Projects/reprojection/calibration/intrinsics/omni_calib.pkl"
with open(OMNI_CALIB_FILE, "rb") as f:
    cam1_calib = pickle.load(f)

images = sorted(os.listdir(OMNI_IMAGES))
mesh_files = sorted(os.listdir(REPROJECTED_MESHES_FOLDER))
assert len(images) == len(mesh_files), "Number of images and meshes do not match"

def view_image(index):
    mesh_file = mesh_files[index]
    img_path = images[index]
    
    mesh = trimesh.load(os.path.join(REPROJECTED_MESHES_FOLDER, mesh_file))
    use_omni, camera_matrix, xi, dist_coeffs = get_camera_params(cam1_calib)
    
    if use_omni:
        vertices_2d, _ = cv2.omnidir.projectPoints(
            np.expand_dims(mesh.vertices, axis=0),
            np.zeros(3),
            np.zeros(3),
            camera_matrix,
            xi,
            dist_coeffs,
        )
        vertices_2d = vertices_2d.swapaxes(0, 1)
    else:
        vertices_2d, _ = cv2.projectPoints(
            mesh.vertices,
            np.zeros(3),
            np.zeros(3),
            camera_matrix,
            dist_coeffs
        )

    img = Image.open(os.path.join(OMNI_IMAGES, img_path))
    img = render_mesh(img, img_path, mesh, vertices_2d, output_dir=None, show_on_screen=False)
    display(img)
    

slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(images)-1,
    step=1,
    description='Image Index:',
    continuous_update=False
)

widgets.interact(view_image, index=slider)

In [6]:
def get_filepaths_modified(root, n):
    # A function both the humor and wham widgets depend on
    cam1_images_path = f"{root}/omni"
    capture_dir = f"{root}/capture{n}/rgb"
    sync_file = f"{root}/synced_filenames_full.txt"
    results_folder = f"{root}/capture{n}/out_capture{n}/results_out/final_results"
    for path in [cam1_images_path, capture_dir, sync_file, results_folder]:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Path {path} does not exist!")
    
    return capture_dir, cam1_images_path, sync_file, results_folder

In [ ]:
# Script that uses the saved non-reprojected HuMoR meshes, reprojects them and then renders them on the omni images.
from reproject.reproject_mesh.reprojection_utils import get_calib_paths

# Define paths
n = 1
ROOT_DIR = "/home/NAS-mountpoint/kinect-omni-ego/2024-01-12/at-unis/lab/sid"
HUMOR_MESH_FILE = os.path.join(ROOT_DIR, f"capture{n}", f"out_capture{n}", "results_out", "final_results", "stage3_results.npz")
OMNI_CALIB_FILE = "/home/sid/Projects/reprojection/calibration/intrinsics/omni_calib.pkl"

CAM0_IMAGES_DIR, CAM1_IMAGES_DIR, SYNC_FILE, results_folder = get_filepaths_modified(ROOT_DIR, n)
KINECT_JSONPATH, OMNI_JSONPATH, CAM0_TO_WORLD_PTH, WORLD_TO_CAM1_PTH = get_calib_paths(ROOT_DIR, use_matlab=False, n=n)

# Load camera calibration
with open(OMNI_CALIB_FILE, "rb") as f:
    camera_calib = pickle.load(f)
use_omni, camera_matrix, xi, dist_coeffs = get_camera_params(camera_calib)

# Load meshes from HUMOR_MESH_FILE
print("[*] Loading meshes from HUMOR_MESH_FILE...")
pred_body = get_meshes(os.path.dirname(HUMOR_MESH_FILE))

# Transform meshes
print("[*] Transforming meshes...")
transformed_meshes = transform_meshes(
    pred_body,
    CAM0_TO_WORLD_PTH,
    WORLD_TO_CAM1_PTH,
    KINECT_JSONPATH,
    OMNI_JSONPATH,
    use_matlab=False
)

# Synchronize meshes with images
print("[*] Synchronizing meshes with images...")
synced_cam1_files, synced_meshes = get_synced_meshes(
    SYNC_FILE,
    CAM0_IMAGES_DIR,
    transformed_meshes,
    n=n
)

# Filter images
images = [os.path.join(CAM1_IMAGES_DIR, img) for img in sorted(os.listdir(CAM1_IMAGES_DIR)) if img in synced_cam1_files]
mesh_files = synced_meshes
assert len(images) == len(mesh_files), "Number of images and meshes do not match"


# Define view_image function
def view_image(index, use_omni=True, camera_matrix=camera_matrix, xi=xi, dist_coeffs=dist_coeffs):
    mesh = mesh_files[index]
    img_path = images[index]
    
    if use_omni:
        vertices_2d, _ = cv2.omnidir.projectPoints(
            np.expand_dims(mesh.vertices, axis=0),
            np.zeros(3),
            np.zeros(3),
            camera_matrix,
            xi,
            dist_coeffs,
        )
        vertices_2d = vertices_2d.swapaxes(0, 1)
    else:
        vertices_2d, _ = cv2.projectPoints(
            mesh.vertices,
            np.zeros(3),
            np.zeros(3),
            camera_matrix,
            dist_coeffs
        )
    
    img = Image.open(img_path)
    img = render_mesh(
        img,
        img_path,
        mesh,
        vertices_2d,
        output_dir=None,
        show_on_screen=False
    )
    display(img)

# Create slider widget
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(images)-1,
    step=1,
    description='Image Index:',
    continuous_update=False
)

# Link the slider to the view_image function
widgets.interact(view_image, index=slider, use_omni=True, camera_matrix=widgets.fixed(camera_matrix), xi=widgets.fixed(xi), dist_coeffs=widgets.fixed(dist_coeffs))

In [9]:
# Script that renders the non-reprojected WHAM meshes on omni, based on the HuMoR script above.
n = 0
ROOT_DIR = "/home/NAS-mountpoint/kinect-omni-ego/2024-01-12/at-unis/lab/sid"
WHAM_MESH_FILE = f'/home/sid/Projects/WHAM/output/demo/2024-01-12_at-unis_lab_sid_capture0/rgb/wham_output.pkl'
BM_PATH = "/home/sid/Projects/humor/body_models/smplh/male/model.npz"
OMNI_CALIB_FILE = "/home/sid/Projects/reprojection/calibration/intrinsics/omni_calib.pkl"

with open(OMNI_CALIB_FILE, "rb") as f:
    camera_calib = pickle.load(f)

use_omni, camera_matrix, xi, dist_coeffs = get_camera_params(camera_calib)
CAM0_IMAGES_DIR, CAM1_IMAGES_DIR, SYNC_FILE, _ = get_filepaths_modified(ROOT_DIR, n)
KINECT_JSONPATH, OMNI_JSONPATH, CAM0_TO_WORLD_PTH, WORLD_TO_CAM1_PTH = get_calib_paths(ROOT_DIR, use_matlab=False, n=n)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open(WHAM_MESH_FILE, "rb") as file:
    wham_output_0 = joblib.load(file)

with open(CAM0_TO_WORLD_PTH, "rb") as f:
    cam0_to_world = make_44(pickle.load(f))
with open(WORLD_TO_CAM1_PTH, "rb") as f:
    world_to_cam1 = make_44(pickle.load(f))

transform = world_to_cam1 @ cam0_to_world
transform[:3, 3] = transform[:3, 3] / 1000.0

mesh_sync = MeshSynchronizer(SYNC_FILE, CAM0_IMAGES_DIR)
mesh_processor = MeshProcessor()
renderer = Renderer(pickle.load(open(OMNI_CALIB_FILE, "rb")))
pred_bodies = mesh_processor.load_meshes(os.path.dirname(WHAM_MESH_FILE))
transformed_meshes = mesh_processor.transform_meshes(pred_bodies, transform)
synced_files, synced_meshes = mesh_sync.get_synced_meshes(transformed_meshes, n)

def view_image_notebook(idx, base_dir, synced_files, meshes=None, renderer=None):
    """Interactive notebook viewer for images and meshes"""
    # Setup matplotlib
    plt.clf()
    fig, ax = plt.subplots(figsize=(12, 8))
    img_path = os.path.join(base_dir, synced_files[idx])
    img = Image.open(img_path)
    img = ImageOps.mirror(img)
    
    # Render meshes if provided
    if meshes:
        curr_meshes = meshes[idx]
        for mesh in curr_meshes:
            vertices_2d = renderer.project_vertices(mesh)
            img = renderer.render_mesh(img, mesh, vertices_2d)
    
    # Display in notebook
    ax.imshow(img)
    ax.axis('off')
    plt.show()

# Create slider widget
slider = widgets.IntSlider(
    min=0,
    max=len(synced_files)-1,
    step=1,
    description='Frame:',
    continuous_update=False
)

# Setup interactive display
widgets.interact(
    view_image_notebook,
    idx=slider,
    base_dir=widgets.fixed(CAM1_IMAGES_DIR),
    synced_files=widgets.fixed(synced_files),
    meshes=widgets.fixed(synced_meshes),
    renderer=widgets.fixed(renderer)
)

2024-12-18 20:19:15,108 - INFO - Loaded sync data with 2576 entries
2024-12-18 20:19:15,109 - INFO - Initialized renderer with camera calibration, using omni: True
2024-12-18 20:19:15,196 - INFO - Sanitizing WHAM output
100%|██████████| 11/11 [00:00<00:00, 11258.50it/s]
2024-12-18 20:19:15,199 - INFO - Collating WHAM meshes for 11 sequences
100%|██████████| 11/11 [00:01<00:00,  7.30it/s]
2024-12-18 20:19:16,711 - INFO - Transforming 3866 meshes
100%|██████████| 3866/3866 [00:02<00:00, 1300.29it/s]
2024-12-18 20:19:20,094 - INFO - Synchronized 2196 meshes with camera files


interactive(children=(IntSlider(value=0, continuous_update=False, description='Frame:', max=2195), Output()), …

<function __main__.view_image_notebook(idx, base_dir, synced_files, meshes=None, renderer=None)>

In [4]:
K0_CALIB_FILE = "/home/sid/Projects/reprojection/calibration/intrinsics/k0_rgb_calib.pkl"
with open(K0_CALIB_FILE, "rb") as f:
    k0_calib = pickle.load(f)
k0_calib

{'intrinsics': array([[713.45902492,   0.        , 617.04302192],
        [  0.        , 714.28289621, 360.85253964],
        [  0.        ,   0.        ,   1.        ]]),
 'distortion': array([[ 0.08663132, -0.14762869, -0.00344924,  0.00165397,  0.06665071]]),
 'xi': None,
 'img_shape': (720, 1280)}

In [ ]:
fx = k0_calib["intrinsics"][0, 0]
fy = k0_calib["intrinsics"][1, 1]
cx = k0_calib["intrinsics"][0, 2]
cy = k0_calib["intrinsics"][1, 2]

In [ ]:
fx, fy, cx, cy

In [ ]:
# Save as txt file with fx fy cx cy
np.savetxt("k0_intrinsics.txt", np.array([fx, fy, cx, cy]), fmt="%.6f")